# Latent Dirichlet Allocation

In [1]:
#import functions and load data
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyLDAvis
import pyLDAvis.lda_model

import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)

wdm = pd.read_csv('../data/wdms/count/worldnewsapi/lemmed/text.csv',index_col=0).iloc[:,4:100].astype(int)
wdm.head()

   ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
    --------------------------------------- 0.0/2.6 MB 991.0 kB/s eta 0:00:03
   --- ------------------------------------ 0.2/2.6 MB 2.8 MB/s eta 0:00:01
   -------- ------------------------------- 0.5/2.6 MB 4.8 MB/s eta 0:00:01
   --------------- ------------------------ 1.0/2.6 MB 5.7 MB/s eta 0:00:01
   ---------------- ----------------------- 1.1/2.6 MB 5.6 MB/s eta 0:00:01
   ---------------- ----------------------- 1.1/2.6 MB 5.6 MB/s eta 0:00:01
   ------------------------ --------------- 1.6/2.6 MB 5.3 MB/s eta 0:00:01
   -------------------------- ------------- 1.7/2.6 MB 5.0 MB/s eta 0:00:01
   -------------------------------- ------- 2.1/2.6 MB 5.4 MB/s eta 0:00:01
   ---------------------------------------- 2.6/2.6 MB 5.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/24.0 MB ? eta -:--:--
    --------------------------------------- 0.4/24.0 MB 10.9 MB/s eta 0:00:03
   - ----------------

C:\Users\Owner\AppData\Local\Temp\ipykernel_225180\56450832.py:8: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  wdm = pd.read_csv('../data/wdms/count/worldnewsapi/lemmed/text.csv',index_col=0).iloc[:,4:100].astype(int)


,said,transgender,people,woman,school,gender,year,trans,state,child,...,place,house,saying,home,added,thing,legal,judge,minor,kid
0,0,46,0,0,0,0,2,0,0,0,...,0,0,1,0,0,0,0,22,0,0
1,0,38,9,6,0,15,6,7,0,1,...,0,0,1,0,0,0,14,0,0,0
2,0,36,21,0,0,2,2,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,5,31,2,0,12,12,10,0,19,8,...,0,0,0,0,3,0,0,0,1,1
4,0,35,35,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
num_topics = 2

lda_model_DH = LatentDirichletAllocation(n_components=num_topics, 
                                         max_iter=100, learning_method='online')
LDA_DH_Model = lda_model_DH.fit_transform(wdm)


print("SIZE: ", LDA_DH_Model.shape)  # (NO_DOCUMENTS, NO_TOPICS)

SIZE:  (3101, 7)


In [16]:
# Let's see how the first document in the corpus looks like in
## different topic spaces
print("First headline...")
print(LDA_DH_Model[0])
print("Sixth headline...")
print(LDA_DH_Model[5])

First headline...
[2.51363031e-04 2.51279955e-04 7.88131612e-02 2.51163353e-04
 2.51186958e-04 9.19930341e-01 2.51504740e-04]
Sixth headline...
[5.14224296e-01 2.34106094e-04 2.30262082e-01 2.34346308e-04
 7.09430470e-03 1.21150626e-02 2.35835802e-01]


In [ ]:
#print(lda_model_DH.components_)


## implement a print function 
## REF: https://nlpforhackers.io/topic-modeling/
def print_topics(model, vectorizer, top_n=10):
    for idx, topic in enumerate(model.components_):
        print("Topic:  ", idx)
      
        print([(vectorizer.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-top_n - 1:-1]])
                        ## gets top n elements in decreasing order
    

####### call the function above with our model and CountV
print_topics(lda_model_DH, MyCountV, 15)



## Print LDA using print function from above
########## Other Notes ####################
#import pyLDAvis.sklearn as LDAvis
#import pyLDAvis
#import pyLDAvis.gensim 
## conda install -c conda-forge pyldavis
#pyLDAvis.enable_notebook() ## not using notebook
#panel = LDAvis.prepare(lda_model_DH, MyDTM_DF, MyCountV, mds='tsne')
#pyLDAvis.show(panel)
#panel = pyLDAvis.gensim.prepare(lda_model_DH, MyDTM, MyCountV, mds='tsne')
#pyLDAvis.show(panel)
##########################################################################

import matplotlib.pyplot as plt
import numpy as np

word_topic = np.array(lda_model_DH.components_)
#print(word_topic)
word_topic = word_topic.transpose()

num_top_words = 15
vocab_array = np.asarray(vocab)

#fontsize_base = 70 / np.max(word_topic) # font size for word with largest share in corpus
fontsize_base = 20

for t in range(num_topics):
    plt.subplot(1, num_topics, t + 1)  # plot numbering starts with 1
    plt.ylim(0, num_top_words + 0.5)  # stretch the y-axis to accommodate the words
    plt.xticks([])  # remove x-axis markings ('ticks')
    plt.yticks([]) # remove y-axis markings ('ticks')
    plt.title('Topic #{}'.format(t))
    top_words_idx = np.argsort(word_topic[:,t])[::-1]  # descending order
    top_words_idx = top_words_idx[:num_top_words]
    top_words = vocab_array[top_words_idx]
    top_words_shares = word_topic[top_words_idx, t]
    for i, (word, share) in enumerate(zip(top_words, top_words_shares)):
        plt.text(0.3, num_top_words-i-0.5, word, fontsize=fontsize_base)
                 ##fontsize_base*share)

plt.tight_layout()
plt.show()




